In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
DATA_DIR = '/kaggle/input/h-and-m-personalized-fashion-recommendations'
os.listdir(DATA_DIR)

# EDA

In [ ]:
articles = pd.read_csv(os.path.join(DATA_DIR, 'articles.csv'))
print(articles.shape)
articles.head()

In [ ]:
# All article_ids are unique
articles['article_id'].value_counts()

## Train transactions

In [ ]:
trx_train = pd.read_csv(os.path.join(DATA_DIR, 'transactions_train.csv'))
print(trx_train.shape)
trx_train.head()

In [ ]:
trx_train['t_dat'] = pd.to_datetime(trx_train['t_dat']).dt.date

In [ ]:
pd.to_datetime(trx_train['t_dat']).describe()

In [ ]:
trx_train['t_dat'].value_counts().sort_index().plot(kind='line', figsize=(20, 5))
plt.axvline(x=pd.to_datetime('2019-09-23'), color='red')
plt.axvline(x=pd.to_datetime('2019-09-30'), color='red')
plt.axvline(x=pd.to_datetime('2018-09-23'), color='red')
plt.axvline(x=pd.to_datetime('2018-09-30'), color='red')

In [ ]:
trx_train['customer_id'].value_counts().describe()

In [ ]:
trx_train = trx_train.merge(articles.loc[:, ['article_id', 'product_type_no']], on='article_id', how='left', copy=False)

In [ ]:
len(trx_train) - trx_train.count()

In [ ]:
(trx_train['article_id'].value_counts()/trx_train.shape[0])[:12]

In [ ]:
trx_train['relevant_period'] = 0
trx_train['relevant_period_2018'] = 0
trx_train['relevant_period_2019'] = 0


trx_train.loc[((trx_train['t_dat']>=pd.to_datetime('2019-09-23')) & (trx_train['t_dat']<=pd.to_datetime('2019-09-29'))), 'relevant_period_2019'] = 1
trx_train.loc[((trx_train['t_dat']>=pd.to_datetime('2018-09-23')) & (trx_train['t_dat']<=pd.to_datetime('2018-09-29'))), 'relevant_period_2018'] = 1

trx_train['relevant_period'] = trx_train['relevant_period_2018'] + trx_train['relevant_period_2019']
trx_train['relevant_period'].value_counts()

In [ ]:
trx_train['prev_week_2019'] = 0
trx_train.loc[((trx_train['t_dat']>=pd.to_datetime('2019-09-15')) & (trx_train['t_dat']<=pd.to_datetime('2019-09-22'))), 'prev_week_2019'] = 1

In [ ]:
trx_train['prev_month_2019'] = 0
trx_train.loc[((trx_train['t_dat']>=pd.to_datetime('2019-08-22')) & (trx_train['t_dat']<=pd.to_datetime('2019-09-22'))), 'prev_month_2019'] = 1

In [ ]:
trx_train['prev_day_2019'] = 0
trx_train.loc[((trx_train['t_dat']==pd.to_datetime('2019-09-22'))), 'prev_day_2019'] = 1

In [ ]:
trx_train['compound_factor'] = trx_train['prev_week_2019'] + trx_train['prev_month_2019'] + trx_train['prev_day_2019']

In [ ]:
vc18 = (trx_train.loc[trx_train['relevant_period_2018']==1, 'article_id'].value_counts()/trx_train['relevant_period_2018'].sum())[:12]
print(vc18.sum())
print(vc18)

In [ ]:
vc19 = (trx_train.loc[trx_train['relevant_period_2019']==1, 'article_id'].value_counts()/trx_train['relevant_period_2019'].sum())[:12]
print(vc19.sum())
print(vc19)

In [ ]:
vc_prevwk19 = (trx_train.loc[trx_train['prev_week_2019']==1, 'article_id'].value_counts()/trx_train['prev_week_2019'].sum())[:12]
print(vc_prevwk19.sum())
print(vc_prevwk19)

In [ ]:
vc_prevmnth19 = (trx_train.loc[trx_train['prev_month_2019']==1, 'article_id'].value_counts()/trx_train['prev_month_2019'].sum())[:12]
print(vc_prevmnth19.sum())
print(vc_prevmnth19)

In [ ]:
vc_prevday19 = (trx_train.loc[trx_train['prev_day_2019']==1, 'article_id'].value_counts()/trx_train['prev_day_2019'].sum())[:12]
print(vc_prevday19.sum())
print(vc_prevday19)

In [ ]:
incount = vc19.index.isin(vc_prevwk19.index)
print(len(incount[incount]))
print(incount)

In [ ]:
incount = vc19.index.isin(vc_prevmnth19.index)
print(len(incount[incount]))
print(incount)

In [ ]:
incount = vc19.index.isin(vc_prevday19.index)
print(len(incount[incount]))
print(incount)

In [ ]:
concat_incount = (vc19.index.isin(vc_prevday19.index))|(vc19.index.isin(vc_prevmnth19.index))|(vc19.index.isin(vc_prevwk19.index))
print(len(concat_incount[concat_incount]))
print(concat_incount)

In [ ]:
compound_factor_pivot = pd.pivot_table(trx_train.loc[trx_train['prev_month_2019']>0], index='article_id', values='compound_factor', aggfunc='sum').sort_values(by='compound_factor', ascending=False)
compound_factor_pivot

In [ ]:
top_ids_by_compound = compound_factor_pivot.index[:12]
comp_incount = (vc19.index.isin(top_ids_by_compound))
print(len(comp_incount[comp_incount]))
print(comp_incount)

In [ ]:
trx_train.head()

### Define recurring product_type_no

In [ ]:
num_purch_df = pd.pivot_table(trx_train, index=['customer_id', 'product_type_no'], values='t_dat', aggfunc=lambda x: len(x.unique())).reset_index()
num_purch_df = pd.pivot_table(num_purch_df, index='product_type_no', values='t_dat', aggfunc='mean').reset_index()
num_purch_df.rename(columns={'t_dat':'mean_purch_num'}, inplace=True)
num_purch_df.head()

In [ ]:
num_purch_df['recurring'] = np.where(num_purch_df['mean_purch_num']>1, 1, 0)
num_purch_df['recurring'].describe()

In [ ]:
num_purch_df.loc[num_purch_df['recurring']==1]

In [ ]:
articles[articles['product_type_no'].isin([252, 265,272])].head()

### Customers

In [ ]:
customers = pd.read_csv(os.path.join(DATA_DIR, 'customers.csv'))
print(customers.shape)
customers.head()

In [ ]:
customers['club_member_status'].value_counts()

In [ ]:
customers['age'].hist(bins=50)

In [ ]:
customers['postal_code'].value_counts()

# Modelling attempts

## Most frequent articles in last periods

In [ ]:
trx_train.head()

In [ ]:
trx_train['last_week_20'] = 0
trx_train.loc[((trx_train['t_dat']>=pd.to_datetime('2020-09-15')) & (trx_train['t_dat']<=pd.to_datetime('2020-09-22'))), 'last_week_20'] = 1

In [ ]:
trx_train['last_month_20'] = 0
trx_train.loc[((trx_train['t_dat']>=pd.to_datetime('2020-08-22')) & (trx_train['t_dat']<=pd.to_datetime('2020-09-22'))), 'last_month_20'] = 1

In [ ]:
trx_train['last_day_20'] = 0
trx_train.loc[((trx_train['t_dat']==pd.to_datetime('2020-09-22'))), 'last_day_20'] = 1

In [ ]:
vc_prevday20 = trx_train.loc[trx_train['last_day_20']==1, 'article_id'].value_counts().index[:12]
vc_prevday20

# Make preds

In [ ]:
sample_submission = pd.read_csv(os.path.join(DATA_DIR, 'sample_submission.csv'))
print(sample_submission.shape)
sample_submission.head()

In [ ]:
base_pred = ''
for i in vc_prevday20:
    base_pred += str(i)+' '
base_pred = base_pred[:-1]
base_pred

In [ ]:
sample_submission['prediction'] = base_pred
sample_submission

In [ ]:
sample_submission.to_csv('/kaggle/working/preds.csv', index=False)